In [ ]:
!pip install mne

In [ ]:
import numpy as np
import mne
from sklearn.preprocessing import normalize
import csv
import pandas as pd
from google.colab import drive
import torch
import os
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from torch.nn import functional as F
from torchsummary import summary

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
print(torch.__version__)
print(device)

1.10.0+cu111
cuda


In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# THIS PATH WILL BE DIFFFERENT FOR U ALL, FIND URS AND COMMENT OTHERS WHEN U USE:
#path = '/content/drive/MyDrive/UPEnn/junior year/cis519 - project/Data/' ###### ADITYA PATH - DO NO DELETE #########
path = '/content/drive/MyDrive/cis519 - project/Data/'
#path = '/content/drive/MyDrive/cis519 - project/new_data/'

In [ ]:
#S001/S001R01.edf

def edf_to_df(edf):
  new_path = path + edf
  edf_name = edf.split('.')[0].split('/')[1]
  edf_csv = edf_name + ".csv"

  edf_read = mne.io.read_raw_edf(new_path)
  header = ','.join(edf_read.ch_names)
  np.savetxt(edf_csv, edf_read.get_data().T, delimiter=',', header=header)

  data = list(csv.reader(open(edf_csv)))
  data = np.array(data)

  columnLabels = data[0]
  data = np.delete(data, 0, axis = 0)
  data = normalize(data, axis=0, norm='l2')

  df = pd.DataFrame(data, columns = [columnLabels])
  df = df.loc[~(df==0).all(axis=1)]

  return df

In [ ]:
def fastfouriertransform(data, fs, threshold):
  dt = 1/fs #samples per second

  df_to_return = []
  for i in range(len(data.columns)):
    mydata = np.array(data[data.columns[i]])

    f = mydata
    n = len(f)
    fhat = np.fft.fft(f,n)
    PSD = fhat * np.conj(fhat) / n
    freq = (1/(dt*n)) * np.arange(n)
    L = np.arange(1, np.floor(n/2), dtype='int')

    indices = PSD > threshold
    PSDclean = PSD * indices
    fhat = indices * fhat
    ffilt = np.fft.ifft(fhat)
    df_to_return.append(ffilt)

  return pd.DataFrame(df_to_return).T

In [ ]:
# "S001/S001R01.edf"
def bigdf_to_small_numpy_lists(edf):

  data = edf_to_df(edf)
  dataCleaned = fastfouriertransform(data, 160, 0.0025)[0:9216]

  list_of_nps = []
  for p in dataCleaned.columns:
    dataCleaned[p] = dataCleaned[p].apply(lambda x: x.real)
    npData = dataCleaned[p].to_numpy()
    reshapes = np.reshape(npData, (96,96))
    list_of_nps.append(reshapes)

  return dataCleaned, list_of_nps

In [ ]:
#all_files_in = os.listdir(path + "S001")

# 96 * 96 = 9216
# dataCleaned = bigdf_to_small_numpy_lists("S001/S001R02.edf")[0:9216]

# for i in dataCleaned.columns:
#   npData = dataCleaned[i].to_numpy()
#   reshapes = np.reshape(npData, (96,96))
#   print(reshapes.shape)

# dataCleaned

In [ ]:

list_of_edf_files = os.listdir(path)

#all_data = []
train_data = []
test_data = []
df_list = []

print(list_of_edf_files)
list_of_edf_files.remove('.ipynb_checkpoints')
list_of_edf_files = sorted(list_of_edf_files)
print(list_of_edf_files)



for i in list_of_edf_files:
  identifyPerson = int(i.split('S')[1])

  all_files_in = os.listdir(path + "/S{}".format(i.split('S')[1]))
  edf_only = [g for g in all_files_in if g.split('.')[1]=='edf']
  counter = 0
  for datasets in range(len(edf_only)):
    datasets +=1
    if (datasets < 10):
      curr_dataset = "{}R0{}".format(i, datasets)
    else:
      curr_dataset = "{}R{}".format(i, datasets)

###### CREATING MINI-BATCHES OF "IMAGE-LIKE" #################################
    dataCleaned, list_of_nps = bigdf_to_small_numpy_lists("{}/{}.edf".format(i,curr_dataset))   ##### LIMIT TOTAL LENGTH OF DF SO CAN GET PERFECT IMAGES OF 776x776
    # mini_np_list = []
    # for j in range(len(dataCleaned)):
    #   if (j % 64 == 0):
    #     new_df = dataCleaned.loc[j:(j+63), :]
    #     mini_np_list.append(new_df.to_numpy())

    # mini_np_capped = mini_np_list[0:150]
    #npversion = np.array(dataCleaned)
    #reshaped = npversion.reshape(776,776)
    #resized = np.resize(reshaped, (128, 128)) 
##############################################################################
    counter = 0
    for eachNP in list_of_nps:
      if (counter % 5 == 0):
        test_data.append((torch.from_numpy(eachNP)[None,:], identifyPerson-1))
      else:
        train_data.append((torch.from_numpy(eachNP)[None,:], identifyPerson-1))
      counter += 1

    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ just wrapped up working on {} ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~".format(curr_dataset))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")


Extracting EDF parameters from /content/drive/MyDrive/cis519 - project/Data/S001/S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ just wrapped up working on S001R01 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Extracting EDF parameters from /content/drive/MyDrive/cis519 - project/Data/S001/S001R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ just wrapped up working on S001R02 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Ex

KeyboardInterrupt: ignored

In [ ]:
import glob
all_files_in = os.listdir()
print(all_files_in)
arr_files = glob.glob('*.csv');
sequences = list()
for i in range(1,len(arr_files)):
    file_path = arr_files[i]
    df = pd.read_csv(file_path, header=0)
    values = df.values
    sequences.append(values)
print(sequences)

In [ ]:
targets = [1] * 14
targets.extend([2]*14)
targets.extend([3]*14)
targets.extend([4]*14)
targets.extend([5]*14)
targets.extend([6]*14)
targets.extend([7]*14)
targets.extend([8]*14)
targets.extend([9]*14)
targets.extend([10]*14)
targets.extend([11]*14)
targets.extend([12]*14)
targets.extend([13]*14)
targets.extend([14]*14)
targets.extend([15]*14)
print(targets)

In [ ]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
pd.Series(len_sequences).describe()

#Padding the sequence with the values in last row to max length
#to_pad = 20000
new_seq = []
new_targets = []
position = 0
for one_seq in sequences:
    if(len(one_seq) == 19680):
      new_targets.append(targets[position])
      new_seq.append(one_seq)
    position += 1
targets = new_targets
len_sequences = []
for one_seq in new_seq:
    len_sequences.append(len(one_seq))
print(len(targets))
pd.Series(len_sequences).describe()

In [ ]:
train_lstm = np.array(new_seq)
train_target = np.array(targets)
print(train_lstm.shape)


In [ ]:
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from tensorflow.keras.optimizers import Adam # - Works
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
model = Sequential()
model.add(LSTM(256, input_shape=(19680, 64)))
model.add(Dense(3, activation='softmax'))
model.summary()

In [ ]:
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model_lstm.pkl', monitor='train_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train_lstm, train_target, epochs=200, batch_size=32, callbacks=[chk], validation_data=(train_lstm,train_target))

In [ ]:
DIMENSION_OF_EACH_IMAGE = 96

In [ ]:
print("Train Data Length: {}, Test Data Length: {}, Percent Test Set: {}".format(
    len(train_data), 
    len(test_data), 
    round((len(test_data)/(len(train_data)+len(test_data))) *100,2)))
      

Train Data Length: 1224, Test Data Length: 312, Percent Test Set: 20.31


In [ ]:
peopleVal = [i[1] for i in train_data]
print(set(peopleVal))

peopleVal = [i[1] for i in test_data]
print(set(peopleVal))

{0, 1}
{0, 1}


In [ ]:
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
test_loader = DataLoader(test_data, batch_size=8, shuffle=False)

In [ ]:
n = [label for input, label in train_loader]
print(set(n))

n = [label for input, label in test_loader]
print(set(n))

{tensor([0, 0, 0, 0, 1, 0, 0, 1]), tensor([0, 1, 1, 1, 1, 1, 1, 1]), tensor([0, 0, 0, 0, 1, 0, 0, 0]), tensor([0, 0, 1, 1, 0, 0, 0, 1]), tensor([1, 1, 0, 0, 1, 0, 1, 1]), tensor([1, 0, 0, 1, 0, 0, 0, 1]), tensor([1, 1, 0, 0, 1, 0, 1, 0]), tensor([0, 0, 0, 0, 0, 1, 0, 1]), tensor([0, 0, 0, 0, 0, 1, 0, 0]), tensor([1, 1, 1, 0, 0, 0, 1, 0]), tensor([0, 1, 1, 1, 0, 0, 0, 1]), tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([1, 1, 0, 0, 0, 0, 0, 1]), tensor([1, 0, 0, 1, 1, 0, 0, 0]), tensor([0, 0, 1, 0, 0, 0, 0, 1]), tensor([0, 0, 0, 0, 1, 0, 1, 1]), tensor([0, 0, 0, 0, 0, 1, 0, 0]), tensor([0, 0, 1, 0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 0, 1, 0, 1]), tensor([0, 0, 1, 0, 1, 0, 0, 0]), tensor([0, 1, 1, 0, 1, 1, 0, 0]), tensor([1, 0, 1, 0, 0, 0, 1, 1]), tensor([1, 0, 0, 0, 1, 0, 1, 1]), tensor([1, 1, 0, 0, 1, 0, 0, 0]), tensor([1, 0, 0, 0, 0, 1, 1, 0]), tensor([0, 0, 0, 1, 0, 0, 0, 0]), tensor([1, 1, 1, 0, 0, 1, 0, 1]), tensor([0, 1, 1, 1, 1, 1, 1, 1]), tensor([1, 1, 1, 0, 0, 0, 1, 0]), tensor([0, 0,

In [ ]:
criterion = torch.nn.CrossEntropyLoss();
#criterion = torch.nn.MSELoss(size_average=False)

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim + 1)
    def forward(self, x):
        print(type(x))
        outputs = self.linear(x.float())
        return outputs

lr_model = LogisticRegression(DIMENSION_OF_EACH_IMAGE*DIMENSION_OF_EACH_IMAGE, 15) # 64x64 images that are beign used to classify 15 people for now

In [ ]:
lr_input_dim = DIMENSION_OF_EACH_IMAGE * DIMENSION_OF_EACH_IMAGE    # TODO: use your input dimension for the Logistic Regression model here
print(lr_model.to(device))
summary(lr_model.to(device), (1, lr_input_dim))

LogisticRegression(
  (linear): Linear(in_features=9216, out_features=16, bias=True)
)
<class 'torch.Tensor'>
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 16]         147,472
Total params: 147,472
Trainable params: 147,472
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.04
Forward/backward pass size (MB): 0.00
Params size (MB): 0.56
Estimated Total Size (MB): 0.60
----------------------------------------------------------------


In [ ]:
lr_optimizer = torch.optim.Adam(lr_model.parameters(), lr=0.001)

In [ ]:
def train_model(model, optimizer, criterion, epochs):
    for child in model.children():
      if hasattr(child, 'reset_parameters'):
        child.reset_parameters()
    model.train()
    model.to(device)  
    for i in range(epochs): 
      running_loss = 0.0;
      correct = 0.0;
      total = 0.0;
      for (data, labels) in train_loader:
        flattened = data.view(-1, DIMENSION_OF_EACH_IMAGE * DIMENSION_OF_EACH_IMAGE);

        flattened_n = flattened.to(device)
        #print(labels)
        labels = labels.to(device)
        #print(labels)
        #print(flattened_n)

        outputs = model(flattened_n)

        optimizer.zero_grad()
        
        #labels = labels.to(torch.float32)
        #outputs = outputs.to(torch.float32)
        
        loss = criterion(outputs, labels);

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        # accuracy
        _, predicted = torch.max(outputs.data, 1)
        #print(predicted)
        #print(labels)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        print("epoch: " + str(i) + " loss: " + str(loss.item()) + " accuracy: " + str(correct / total))

      loss_list.append(loss.item())
      accuracy_list.append(correct / total)

    final_training_accuracy = correct / total    # TODO: Set this to final training accuracy
    final_training_loss = loss.item()        # TODO: Set this to final testing accuracy
    return final_training_loss, final_training_accuracy

In [ ]:
loss_list = []
accuracy_list = [] 
lr_training_loss, lr_training_accuracy = train_model(lr_model, lr_optimizer, criterion,100)

#print("Logistic Regression - the training loss is ", str(lr_training_loss))
#print("Logistic Regression - the training accuracy is ", str(lr_training_accuracy))

In [ ]:
from matplotlib import pyplot as plt
# TODO: Plot a graph of your training accuracies and training loss vs epochs! (5 Points)
# You can make use of the `loss_list` and `accuracy_list` variables defined above.
# Remember to include a legend for the curves denoting whether it is loss or accuracy.
# Please make sure the plot is present when your submit this notebook, 
# otherwise no points would be given for this part!

# Loss
plt.plot(loss_list, label='Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(framealpha=1, frameon=True);
plt.show()
plt.plot(accuracy_list, label='Accuracy')
plt.legend(framealpha=1, frameon=True);
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()


In [ ]:
def test_model(model, criterion):
    model.eval()
    total = 0;
    correct = 0;
    for (data, labels) in test_loader:
        data, labels = data.to(device), labels.to(device)
        
        flattened = data.view(-1, DIMENSION_OF_EACH_IMAGE * DIMENSION_OF_EACH_IMAGE);

        flattened_n = flattened.to(device)
        labels = labels.to(device)

        outputs = model(flattened_n)

        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)

        loss = criterion(outputs, labels) # this is the average loss for one batch of inputs
        loss.backward()

        correct += (predicted == labels).sum().item()

        print("loss: " + str(loss.item()) + ", accurayc: " + str(correct / total))

    testing_loss = loss.item()      # TODO: Set this to your testing loss
    testing_accuracy = correct / total     # TODO: Set this to your testing accuracy
    return  testing_loss, testing_accuracy

In [ ]:
lr_testing_loss, lr_testing_accuracy = test_model(lr_model, criterion)

print("Logistic Regression - the testing loss is ", str(lr_testing_loss))
print("Logistic Regression - the testing accuracy is ", str(lr_testing_accuracy))

# FNN Model

In [ ]:
# TODO: Define and initialize an FNN model, save it in a variable called `fnn_model`
import torch.nn.functional as F
import torch
import torch.nn as nn
fnn_model = nn.Sequential(
    nn.Linear(DIMENSION_OF_EACH_IMAGE*DIMENSION_OF_EACH_IMAGE, 500),
    nn.ReLU(),
    nn.Linear(500, 500),
    nn.ReLU(),
    nn.Linear(500, 500),
    nn.ReLU(),
    nn.Linear(500, 16)
)

In [ ]:
fnn_input_dim = DIMENSION_OF_EACH_IMAGE * DIMENSION_OF_EACH_IMAGE           # TODO: use your input dimension for the FNN model here
summary(fnn_model.to(device), (1, fnn_input_dim))

In [ ]:
def train_model_fnn(model, optimizer, criterion, epochs=10):
    for child in model.children():
      if hasattr(child, 'reset_parameters'):
        child.reset_parameters()
    model.train()
    model.to(device)  
    for i in range(epochs): 
      running_loss = 0.0;
      correct = 0.0;
      total = 0.0;
      for (data, labels) in train_loader:
        flattened = data.view(-1, DIMENSION_OF_EACH_IMAGE * DIMENSION_OF_EACH_IMAGE);

        flattened_n = flattened.to(device)
        #print(labels)
        labels = labels.to(device)
        #print(labels)
        #print(flattened_n)

        outputs = model(flattened_n.float())

        optimizer.zero_grad()
        
        #labels = labels.to(torch.float32)
        #outputs = outputs.to(torch.float32)
        
        loss = criterion(outputs, labels);

        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        print("epoch " + str(i))
        # accuracy
        _, predicted = torch.max(outputs.data, 1)
        #print(predicted)
        #print(labels)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        print("loss: " + str(loss.item()) + ",lllll accurayc: " + str(correct / total))

      loss_list.append(loss.item())
      accuracy_list.append(correct / total)

    final_training_accuracy = correct / total    # TODO: Set this to final training accuracy
    final_training_loss = loss.item()        # TODO: Set this to final testing accuracy
    return final_training_loss, final_training_accuracy

In [ ]:
# Train the FNN model on the training set using the `train_model` function
loss_list = []
accuracy_list = []
fnn_epochs = 15               # TODO: set the number of epochs
fnn_optimizer = torch.optim.Adam(fnn_model.parameters(), lr=0.0001)             # TODO: initialize the optimizer
fnn_training_loss, fnn_training_accuracy = train_model_fnn(fnn_model, fnn_optimizer, criterion, fnn_epochs)

print("Feedforward Neural Network - the training loss is ", str(fnn_training_loss))
print("Feedforward Neural Network - the training accuracy is ", str(fnn_training_accuracy))

In [ ]:
from matplotlib import pyplot as plt
plt.plot(accuracy_list)
plt.title("Accuracy for FNN")
plt.show()
plt.plot(loss_list)
plt.title("Loss for FNN")
plt.show()

In [ ]:
def fnn_test_model(model, criterion):
    model.eval()
    total = 0;
    correct = 0;
    for (data, labels) in test_loader:
        data, labels = data.to(device), labels.to(device)
        
        flattened = data.view(-1, DIMENSION_OF_EACH_IMAGE * DIMENSION_OF_EACH_IMAGE);

        flattened_n = flattened.to(device)
        labels = labels.to(device)

        outputs = model(flattened_n.float())

        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)

        loss = criterion(outputs, labels) # this is the average loss for one batch of inputs
        loss.backward()

        correct += (predicted == labels).sum().item()

        print("loss: " + str(loss.item()) + ", accurayc: " + str(correct / total))

    testing_loss = loss.item()      # TODO: Set this to your testing loss
    testing_accuracy = correct / total     # TODO: Set this to your testing accuracy
    return  testing_loss, testing_accuracy

In [ ]:
# Compute performance on the test set using the test_model function created before
fnn_testing_loss, fnn_testing_accuracy = fnn_test_model(fnn_model, criterion)

print("Feedforward Neural Network - the testing loss is ", str(fnn_testing_loss))
print("Feedforward Neural Network - the testing accuracy is ", str(fnn_testing_accuracy))

# CNN Model

In [ ]:
# TODO: Define and initialize a CNN model, save it in a variable called `cnn_model`
import torch
import torch.nn as nn

class CNN2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(                                # 
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(2,2), stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) ## output beofre maxpool = 48 x 48 x 6, so then after maxpool 24 x 24 x 6

        self.layer2 = nn.Sequential(                                
            nn.Conv2d(in_channels=6, out_channels=6, kernel_size=(3,3), stride=1),
            nn.ReLU()) ## 22 x 22 x 6

        self.layer3 = nn.Sequential(                                
            nn.Conv2d(in_channels=6, out_channels=6, kernel_size=(3,3), stride=1),
            nn.ReLU()) ## 20 x 20 x 6

        self.layer4 = nn.Sequential(                                
            nn.Conv2d(in_channels=6, out_channels=6, kernel_size=(3,3), stride=1),
            nn.ReLU()) ## 18 x 18 x 6

        self.layer5 = nn.Sequential(                                
            nn.Conv2d(in_channels=6, out_channels=6, kernel_size=(3,3), stride=1),
            nn.ReLU()) ## 16 x 16 x 6

        self.layer6 = nn.Sequential(                                
            nn.Conv2d(in_channels=6, out_channels=6, kernel_size=(3,3), stride=1),
            nn.ReLU()) ## 14 x 14 x 6

        self.layer7 = nn.Sequential(                                
            nn.Conv2d(in_channels=6, out_channels=6, kernel_size=(3,3), stride=1),
            nn.ReLU()) ## 12 x 12 x 6

        #self.drop_out = nn.Dropout()
        
        self.fc1 = nn.Linear(12 * 12 * 6, 250) # so instead of 7 * 7 * 64, i have 194 * 194 * 64
        self.fc2 = nn.Linear(250, 100) # so instead of 7 * 7 * 64, i have 194 * 194 * 64
        self.fc3 = nn.Linear(100, 2)
        self.sig = nn.Sigmoid()
    
    def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = self.layer3(out)
      out = self.layer4(out)
      out = self.layer5(out)
      out = self.layer6(out)
      out = self.layer7(out)
      out = out.reshape(out.size(0), -1)
      #out = self.drop_out(out)
      out = self.fc1(out)
      out = self.fc2(out)
      out = self.fc3(out)
      out = self.sig(out)
      return out

cnn_model = CNN2()

In [ ]:
# def train_cnn2(model, optimizer, criterion, epochs):
#     for child in model.children():
#       if hasattr(child, 'reset_parameters'):
#         child.reset_parameters()
#     model.to(device)
#     model.train()
#     total = 0
#     correct = 0
#     for epoch in range(15):
#       running_loss = 0.0
#       for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         inputs = inputs.double()
#         inputs = inputs.unsqueeze(1)
#         #inputs = inputs.float()
#         #print(inputs.shape)

#         outputs = model(inputs)  

#         optimizer.zero_grad()
#         loss = criterion(outputs, labels) # this is the average loss for one batch of inputs
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#     # TODO: Implement your training loop as specified in the description

#       x = correct/total
#       accuracy_list.append(x)
#       y = running_loss/len(train_loader) 
#       loss_list.append(y)
#       print("epoch: {}, loss: {}, accuracy: {}".format(round(epoch,5), round(y,5), round(x,5)))
#     final_training_accuracy = correct/total     # TODO: Set this to final training accuracy
#     final_training_loss = running_loss/len(train_loader)         # TODO: Set this to final testing accuracy
#     return final_training_loss, final_training_accuracy

In [ ]:
# TODO: Define and initialize a CNN model, save it in a variable called `cnn_model`
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import Sigmoid
from torch.nn import LogSoftmax
from torch import flatten
import torch
import torch.nn as nn
import random
random.seed(23)

class CNN(nn.Module):
    def __init__(self):
        # call the parent constructor
        super().__init__()
        # initialize first set of CONV => RELU => POOL layers
        self.layer1 = nn.Sequential(                                # 
            nn.Conv2d(in_channels=1, out_channels=4, kernel_size=(5,5), stride=2)) ## output beofre maxpool = 48 x 48 x 6, so then after maxpool 24 x 24 x 6
        
        self.fc1 = nn.Linear(46, 2)

    
    def forward(self, x):
      out = self.layer1(x)
      out = self.fc1(out)
      return out

# def weight_init(m):
#     if isinstance(m, nn.Conv2d):
#       nn.init.xavier_normal_(m.weight, gain=nn.init.calculate_gain('relu'))
#       nn.init.zeros_(m.bias)
#     elif isinstance(m, nn.Linear):
#       nn.init.xavier_normal_(m.weight)
#       nn.init.zeros_(m.bias)

cnn_model = CNN()
# cnn_model.apply(weight_init)

AttributeError: ignored

In [ ]:
summary(cnn_model.to(device), (1, DIMENSION_OF_EACH_IMAGE, DIMENSION_OF_EACH_IMAGE))

RuntimeError: ignored

In [ ]:
def train_cnn(model, optimizer, criterion, epochs):
    for child in model.children():
      if hasattr(child, 'reset_parameters'):
        child.reset_parameters()
    model.to(device)
    model.train()
    total = 0
    correct = 0
    for epoch in range(epochs):
      running_loss = 0.0
      for inputs, labels in train_loader:
        #labels = labels.unsqueeze(dim = 0)
        inputs, labels = inputs.to(device), labels.to(device)

        #print(labels)
        inputs = inputs.float()
        #print(inputs.shape)
        inputs = inputs.unsqueeze(0)
        #print(inputs.shape)
        #inputs = inputs.permute(1, 0, 2, 3)
        #print(inputs.dtype)
        #print(inputs.shape)

        #inputs = inputs.double()
        #print(labels)
        

        outputs = model(inputs)
        #outputs = torch.argmax(outputs, dim=2)
        labels = labels.float()
        optimizer.zero_grad()
        loss = criterion(outputs, labels) # this is the average loss for one batch of inputs
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        print(outputs.data)
        #print("Predicted: {}, Label: {}".format(predicted.item(), labels.item()))
        #print("pred: {}".format(predicted.numpy))
        #print("labels: {}".format(labels.numpy()))
        total += labels.size(0)
        correct += (predicted.argmax(0) == labels).type(torch.float).sum().item()
    # TODO: Implement your training loop as specified in the description

      x = correct/total
      accuracy_list.append(x)
      y = running_loss/len(train_loader) 
      loss_list.append(y)
      print("epoch: {}, loss: {}, accuracy: {}".format(round(epoch,5), round(y,5), round(x,5)))
    final_training_accuracy = correct/total     # TODO: Set this to final training accuracy
    final_training_loss = running_loss/len(train_loader)         # TODO: Set this to final testing accuracy
    return final_training_loss, final_training_accuracy

In [ ]:
criterion = torch.nn.MSELoss();
loss_list = []
accuracy_list = []
cnn_epochs = 15                       # TODO: set the number of epochs
cnn_optimizer = torch.optim.SGD(cnn_model.parameters(), lr=0.00001)                  # TODO: initialize the optimizer
cnn_model = cnn_model.float()
cnn_training_loss, cnn_training_accuracy = train_cnn(cnn_model, cnn_optimizer, criterion, cnn_epochs)

print("Convolutional Neural Network - the training loss is ", str(cnn_training_loss))
print("Convolutional Neural Network - the training accuracy is ", str(cnn_training_accuracy))

RuntimeError: ignored

In [ ]:
from matplotlib import pyplot as plt
# TODO: Plot a graph of your training accuracies and training loss vs epochs! (5 Points)
# You can make use of the `loss_list` and `accuracy_list` variables defined above.
# Remember to include a legend for the curves denoting whether it is loss or accuracy.
# Please make sure the plot is present when your submit this notebook, 
# otherwise no points would be given for this part!
plt.plot(accuracy_list)
plt.title("Accuracy")
plt.show()
plt.plot(loss_list)
plt.title("Loss")
plt.show()

In [ ]:
def test_cnn(model, criterion):
    model.eval()
    total = 0
    correct = 0
    running_loss = 0.0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        images = images.double()
        images = images.unsqueeze(0)
        images = images.permute(1, 0, 2, 3)


        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    testing_loss = running_loss/len(test_loader)         # TODO: Set this to your testing loss
    testing_accuracy = correct/total     # TODO: Set this to your testing accuracy
    return  testing_loss, testing_accuracy

cnn_testing_loss, cnn_testing_accuracy = test_cnn(cnn_model, criterion)

print("Convolutional Neural Network - the testing loss is ", str(cnn_testing_loss))
print("Convolutional Neural Network - the testing accuracy is ", str(cnn_testing_accuracy))

# RNN Model (Using Keras)

Loading Data

In [ ]:
train_data = torch.load(path + "train_loader.pkl")
test_data = torch.load(path + "test_loader.pkl")

Pre-processing

Model

In [ ]:
import torch
import torch.nn as nn
from keras.layers import Dropout, Masking
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from tensorflow.keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

model = Sequential()

# Recurrent layer
model.add(LSTM(256, input_shape=(19680, 1)))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_words, activation='softmax'))

KeyboardInterrupt: ignored

In [ ]:
# Compile the model
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=128, callbacks=[chk], validation_data=(validation,validation_target))

In [ ]:
model = load_model('best_model.pkl')

from sklearn.metrics import accuracy_score
test_preds = model.predict_classes(test)
accuracy_score(test_target, test_preds)

# Code for testing stuff - this one was to help me figure out FFT

In [ ]:
fs = 160                                # Sampling rate (512 Hz)
highPassFilter = 1
lowPassFilter = 100

data = mydata

import matplotlib.pyplot as plt

for i in range(2):
  mydata = np.array(dfForThisPerson[dfForThisPerson.columns[i]])
  #filter1highpass = [g for g in mydata if (((1/g) > highPassFilter or (1/g) < -highPassFilter) and g!=0)]
  #filter2lowpass = [g for g in filter1highpass if (1/g) < lowPassFilter and (1/g) > -lowPassFilter and g!=0]

  dt = 1/fs #samples per second

  f = mydata
  n = len(f)
  fhat = np.fft.fft(f,n)
  PSD = fhat * np.conj(fhat) / n
  freq = (1/(dt*n)) * np.arange(n)
  L = np.arange(1, np.floor(n/2), dtype='int')

  indices = PSD > 0.0025
  PSDclean = PSD * indices
  fhat = indices * fhat
  ffilt = np.fft.ifft(fhat)

  # NO NEED FOR HIGH PASS FILTER, SAME RESULT, LOW PASS FILTER REDUCES DATA TOO MUCH
  plt.title("FFT")
  plt.plot(freq[L], PSD[L])
  left, right = plt.xlim()
  plt.xlim(left, 20)
  plt.show()

  plt.title("Uncleaned vs. Cleaned Data")
  plt.plot(mydata, color='k', label='unclean')
  plt.plot(ffilt, color='r', label='clean')
  plt.legend()
  plt.show()

  plt.plot(filter2lowpass)
  plt.show()




In [ ]:
# TODO: Define and initialize a CNN model, save it in a variable called `cnn_model`
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(                                # 
            nn.Conv2d(in_channels=1, out_channels=4, kernel_size=6, stride=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) ## output beofre maxpool = 46 x 46 x 4, so then after maxpool 23 x 23 x 4

        self.layer2 = nn.Sequential(                                # 
            nn.Conv2d(in_channels=4, out_channels=4, kernel_size=4, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) ## output beofre maxpool = 20 x 20 x 4, so then after maxpool 10 x 10 x 4
        
        #self.drop_out = nn.Dropout()
        
        self.fc1 = nn.Linear(10 * 10 * 4, 100) # so instead of 7 * 7 * 64, i have 194 * 194 * 64
        self.fc2 = nn.Linear(100, 15)
    
    def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = out.reshape(out.size(0), -1)
      #out = self.drop_out(out)
      out = self.fc1(out)
      out = self.fc2(out)
      return out

cnn_model = CNN()